In [ ]:
import os
import pandas as pd
import sqlite3

from query_insights.utils import load_config,load_data_dictionary

data_config_path = "../../configs/local/data_config.yaml"

config = load_config(cfg_file=data_config_path)

# Connect to the SQLite database
conn = sqlite3.connect(config.db_params.sqlite_database_path)

# Specify the name of the table to extract
table_name = 'invoice_data'

# Get the list of columns with text values from the SQLite table
text_columns = []
cursor = conn.execute(f"PRAGMA table_info({table_name})")
for column_info in cursor:
    if column_info[2] == 'TEXT':
        text_columns.append(column_info[1])

# Load the data dictionary from json file
# with open(r"C:\Users\chetan.subramani\Downloads\yf_data\invoice_data.json", 'r') as f:
#     data_dict = json.load(f)

data_dict = load_data_dictionary(os.path.join(config.path.data_dictionary_path, f"{table_name}.json"))
    
# Loop through each string column in DataFrame and update the data dictionary
for col in text_columns:
    unique_values = pd.read_sql(f"SELECT distinct {col} FROM {table_name}", conn).values.ravel().tolist()
    if len(unique_values) < 10:
        # unique_values = df[col].unique().tolist()
        for column in data_dict['columns']:
            if column['name'] == col:
                print(col)
                column['unique_values'] = unique_values
                break

# Save the updated data dictionary to json file
# with open('yf_data_dict.json', 'w') as f:
#     json.dump(data_dict, f, indent=4)
